## Analyse exploratoire et nettoyage de `name.basics`

### Introduction
Les étapes suivantes visent à analyser, nettoyer et préparer la table `name.basics` pour une utilisation ultérieure. 
Nous allons examiner chaque colonne, effectuer les transformations nécessaires et proposer des recommandations pour les prochaines étapes.

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

### Étape 1 : Chargement des données

In [2]:
# Importation du DataSet :
df_name_basics = pd.read_csv('../gitignore/name.basics.tsv', sep= '\t')

- #### Aperçu rapide des données

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


### Étape 2 : Informations générales sur les données

- #### Aperçu rapide des données

In [6]:
df_name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


- #### Identification des valeurs manquantes

In [7]:
df_name_basics.isna().sum()

nconst               0
primaryName          9
birthYear            0
deathYear            0
primaryProfession    0
knownForTitles       0
dtype: int64

In [8]:
manquant_primary_name = df_name_basics[df_name_basics['primaryName'].isna()]

print("Lignes avec des valeurs manquantes dans `primaryName` :")
print(manquant_primary_name)

# Vérification du nombre total de valeurs manquantes
print(f"Nombre total de valeurs manquantes : {manquant_primary_name.shape[0]}")

Lignes avec des valeurs manquantes dans `primaryName` :
             nconst primaryName birthYear deathYear        primaryProfession  \
100967    nm0106021         NaN        \N        \N                    actor   
1786541  nm10927938         NaN        \N        \N  art_department,composer   
4927108  nm14292439         NaN        \N        \N                       \N   
4962416  nm14334668         NaN        \N        \N                 director   
4993716  nm14381073         NaN        \N        \N          director,writer   
5877414  nm15383643         NaN        \N        \N                 composer   
6311127  nm15901107         NaN        \N        \N                    actor   
7029620  nm16712594         NaN        \N        \N           actor,composer   
8387433   nm3177777         NaN        \N        \N                  actress   

                knownForTitles  
100967              tt21610866  
1786541  tt16401648,tt10837972  
4927108                     \N  
4962416    

- #### Remplacement des valeurs '\N' par NaN pour faciliter le traitement

In [ ]:
df_name_basics = df_name_basics.replace(r'\\N', pd.NA, regex=True)

KeyboardInterrupt: 


# Remplacement des valeurs '\N' par NaN pour faciliter le traitement
df_name_basics = df_name_basics.replace(r'\\N', np.nan, regex=True)

# %% [markdown]
# ## Étape 3 : Analyse des colonnes `birthYear` et `deathYear`
#
# ### Conversion des types de données

# %%
# Conversion des colonnes en types numériques pour analyse
df_name_basics['birthYear'] = pd.to_numeric(df_name_basics['birthYear'], errors='coerce')
df_name_basics['deathYear'] = pd.to_numeric(df_name_basics['deathYear'], errors='coerce')

# %% [markdown]
# ### Exploration des données numériques

# %%
# Aperçu statistique
df_name_basics[['birthYear', 'deathYear']].describe()

# Calcul de la durée de vie pour les entrées valides
df_name_basics['lifespan'] = df_name_basics['deathYear'] - df_name_basics['birthYear']

# Distribution des durées de vie
df_name_basics['lifespan'].describe()

# %% [markdown]
# ## Étape 4 : Analyse de la colonne `primaryProfession`
#
# ### Transformation en liste et explosion

# %%
# Conversion des professions en liste
df_name_basics['primaryProfession'] = df_name_basics['primaryProfession'].apply(
    lambda x: x.split(',') if isinstance(x, str) else []
)

# Explosion des professions
df_exploded_profession = df_name_basics.explode('primaryProfession', ignore_index=True)

# %% [markdown]
# ### Analyse des professions

# %%
# Comptage des professions les plus fréquentes
df_exploded_profession['primaryProfession'].value_counts().head(10)

# %% [markdown]
# ## Étape 5 : Analyse de la colonne `knownForTitles`
#
# ### Transformation en liste et explosion

# %%
# Conversion des titres en liste
df_name_basics['knownForTitles'] = df_name_basics['knownForTitles'].apply(
    lambda x: x.split(',') if isinstance(x, str) else []
)

# Explosion des titres
df_exploded_titles = df_name_basics.explode('knownForTitles', ignore_index=True)

# %% [markdown]
# ### Analyse des titres

# %%
# Comptage des titres les plus fréquents
df_exploded_titles['knownForTitles'].value_counts().head(10)

# %% [markdown]
# ## Étape 6 : Visualisation des données
#
# ### Distribution des professions les plus fréquentes

# %%
# Préparation des données pour la visualisation
top_professions = df_exploded_profession['primaryProfession'].value_counts().head(10)

# Création du graphique
plt.figure(figsize=(10, 6))
top_professions.plot(kind='bar', color='skyblue')
plt.title("Professions les plus fréquentes", fontsize=16)
plt.xlabel("Profession", fontsize=12)
plt.ylabel("Nombre de personnes", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# %% [markdown]
# ### Distribution des années de naissance

# %%
# Histogramme des années de naissance
plt.figure(figsize=(10, 6))
df_name_basics['birthYear'].dropna().plot(kind='hist', bins=50, color='coral', alpha=0.8)
plt.title("Distribution des années de naissance", fontsize=16)
plt.xlabel("Année de naissance", fontsize=12)
plt.ylabel("Nombre de personnes", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# %% [markdown]
# ## Étape 7 : Recommandations pour la suite
#
# ### Nettoyage des données
#
# - Conserver les colonnes importantes : 
#   - `nconst` : clé unique pour jointures
#   - `primaryName` : nom principal
#   - `knownForTitles` : titres associés
#   - `primaryProfession` : professions principales
#
# - Nettoyer les colonnes `birthYear` et `deathYear` :
#   - Supprimer ou imputer les valeurs manquantes si nécessaire.
#   - Vérifier les valeurs aberrantes (ex. dates anachroniques).
#
# - Normaliser les listes dans `primaryProfession` et `knownForTitles` pour faciliter les analyses croisées.

# %% [markdown]
# ### Préparation pour les jointures avec d'autres tables
#
# - Utiliser `nconst` pour joindre avec la table `title.principals`.
# - Exploiter `knownForTitles` pour analyser les contributions majeures des individus dans les films ou séries.
